<a href="https://colab.research.google.com/github/niklaust/Deep_Learning/blob/main/PyTorch_for_Deep_Learning_notebook_of_nikluast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reference**
Ian Pointer. (2019). *Programming PyTorch For Deep Learning Creating and Deploying Deep Learning Application*. O'Reilly

github:niklaust

start 20230220

<h1><center><b>Programming PyTorch for Deep Learning</b></center></h1>

## **What is Deep Learning?**

**A machine learning technique** that **uses multiple and numerous layers of nonlinear** transforms to progressively extract features from raw input

**A technique to solve problems** by providing the inputs and desired outputs and letting the computer find the solution, normally **using a neural network.**

# <center><b>Chapter 1. Getting Started with PyTorch</b></center>

**GPU** is The **heart** of every deep learning box. It is going to **power the majority of PyTorch's calculations.**

## **Getting Start with PyTorch!** 

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.rand(2, 2))

True
tensor([[0.8933, 0.0782],
        [0.8609, 0.6029]])


## **Tensors**

A **tensor is both a container for numbers as well as a set of rules** that define transformations between tensors that produce new tensors.

It's easier to think **tensors as multidimensional arrays.**

In [ ]:
x = torch.tensor([[0,0,1],[1,1,1],[0,0,0]])
x

tensor([[0, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

Change an element in a tnsor by using standard Python indexing:

In [ ]:
x[0][0] = 5
x

tensor([[5, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

In [ ]:
torch.zeros(2,2)

tensor([[0., 0.],
        [0., 0.]])

In [ ]:
torch.ones(3,3)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
torch.ones(1,2) + torch.ones(1,2)

tensor([[2., 2.]])

`item`: pull out the value 

In [ ]:
torch.manual_seed(42)

torch.rand(1).item()

0.8822692632675171

`to`: copy between devices 

In [ ]:
import torch

In [ ]:
cpu_tensor = torch.rand(2)
cpu_tensor.device

device(type='cpu')

In [ ]:
gpu_tensor = cpu_tensor.to("cuda")
gpu_tensor.device

device(type='cuda', index=0)

### **Tensor Operations**

In [ ]:
torch.manual_seed(42)

a = torch.rand(2,2)
a

tensor([[0.8823, 0.9150],
        [0.3829, 0.9593]])

In [ ]:
print(a.max())             # torch.rand(2,2).max()

tensor(0.9593)


In [ ]:
print(a.max().item())      # torch.rand(2,2).max().item()

0.9593056440353394


`type`: to see the element type in the tensor
`dtype`: to change the type of a tensor

In [ ]:
long_tensor = torch.tensor([[0,0,1],[1,1,1],[0,0,0]])
long_tensor.type()

'torch.LongTensor'

In [ ]:
float_tensor = torch.tensor([[0,0,1],[1,1,1],[0,0,0]]).to(dtype=torch.float32)
float_tensor.type()

'torch.FloatTensor'

appended underscore `_`: save memory, look to see if an in-place function is defined 

In [ ]:
torch.manual_seed(42)

random_tensor = torch.rand(2,2)
random_tensor.log2()

tensor([[-0.1807, -0.1282],
        [-1.3851, -0.0599]])

In [ ]:
random_tensor.log2_()

tensor([[-0.1807, -0.1282],
        [-1.3851, -0.0599]])

**reshape a tensor:**

`view`:  **operates as a view on the original tensor**, so if the underlying data is changed, the view will change too (and vice versa). However, it can throw errors if the required view is not contiguous. It doesn't share the same block of memory it would occupy if a new tensor of  the required shape was created from scratch, you have to call `tensor.contiguous()` before you can use `view()`.

`reshape`: to reshape a tensor

In [ ]:
torch.manual_seed(42)

flat_tensor = torch.rand(784)
print(flat_tensor.shape)                         # 1*28*28 = 784
viewed_tensor = flat_tensor.view(1, 28, 28)
print(viewed_tensor.shape)

torch.Size([784])
torch.Size([1, 28, 28])


In [ ]:
torch.manual_seed(42)

reshaped_tensor = flat_tensor.reshape(1, 28, 28)
print(reshaped_tensor.shape)

torch.Size([1, 28, 28])


In [ ]:
# the reshpaed tensor's shape has to have the same number of 
# total elements as the original. 

try:
  flat_tensor.reshape(3, 28, 28)
except:
  print("RuntimeError: shape '[3, 28, 28]' is invalid for input of size 784")

RuntimeError: shape '[3, 28, 28]' is invalid for input of size 784


**Rearrange the dimensions of a tensor.**

`permute` : rearrange the dimensions of a tensor

You will likely come across this with images, which often are stored as `[height, width, channel]` tensors, but PyTorch prefers to deal with these in a `[channel, height, width]` you can use `permute()` to deal with these in a fairly straightforward manner:

In [ ]:
torch.manual_seed(42)

hwc_tensor = torch.rand(640, 480, 3)            # [height, width, channel]
print(hwc_tensor.shape)
chw_tensor = hwc_tensor.permute(2,0,1)          # [channel, height, width]
print(chw_tensor.shape)

torch.Size([640, 480, 3])
torch.Size([3, 640, 480])


### **Tensor Broadcasting**

Boradcasting allows you to **perform operations between a tensor and a small tensor.** You can broadcast across two tensors if, starting backward from their trailng dimensions:

* The two dimensions are equal.
* One of the dimensions is 1.



# <center><b>Chapter 2. Image Classification with PyTorch</b></center>

## **Our Classification Problem**

Building a simple classifier that can tell the difference between fish and cats.

## **Traditional Challenges**

**Writing a set of rules describing** that a cat has a tail, or that a fish has scales, and **apply those rules to an image to determine**.

We need a lot of pictures of fish and cats. to train the neural network.

We will use ImageNet, a standard collection of images used to train neural networks..

PyTorch needs a way to determine what is a cat and what is a fish. We use a label attached to the data, and training in this manner is called supervised learning.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

In [ ]:
import zipfile

# Specify the path to the ZIP file
zip_path = '/content/images.zip'

# Extract the contents of the ZIP file to a folder named "images"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/images')

## **Data Loaders**

Loading and converting data into formats that are ready for training

The two main conventions of interacting with data are datasets and data loaders.

* A dataset is a Python class that allows us to get the data we're supplying to the neural network.
* A data loader is what feeds data from the dataset into the network.

## **Building a Training Dataset, Validation and Test Datasets**

`torchvision` package in cludes a class called `ImageFolder` providing our images are in a structure where each directory is label

In [ ]:
import torch

In [ ]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [ ]:
import torchvision 
from torchvision import transforms

img_transforms = transforms.Compose([
        transforms.Resize((64, 64)),                      # scale to the same resolution 64x64
        transforms.ToTensor(),                            # take image data and turn it into a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalizing
                         std=[0.229, 0.224, 0.225])
])

**Dataset Types**

* **Training set** : Used in the training pass to update the model
* **Validation set** : Used to evaluate how the model is generalizing to the problem domain, rather than fitting to the training data; not used to update the model directly
* **Test set** : A final dataset that provides a final evaluation of the model's performance after training is complete

In [ ]:
# Training set
train_data_path = "/content/images/train"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,
                                              transform=img_transforms, 
                                              is_valid_file=check_image)

In [ ]:
# Validation set
val_data_path = "/content/images/val"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,
                                            transform=img_transforms, 
                                            is_valid_file=check_image)

In [ ]:
# Test set
test_data_path = "/content/images/test"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,
                                             transform=img_transforms,
                                             is_valid_file=check_image) 

`batch_size` : tell **how many images will go through the network before we train and update it**, in theory, set the `batch_size` to the number of image in the test and training sets so the network sees every image before it updates. In practice, we tend not ot do this because smaller batches (more commonly known as mini-batches in the literature) require less memory than having to store all the information about every image in the dataset, and the smaller batch size ends up making training faster as we're updating our network much more quickly.


In [ ]:
# build our data loaders 
batch_size=64           
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

## **Creating a First Model, SimpleNet**

SimpleNet has three linear layers and ReLu activations between them. 

In [ ]:
class SimpleNet(nn.Module):

  def __init__(self):
    super(SimpleNet, self).__init__()
    self.fc1 = nn.Linear(12288, 84)
    self.fc2 = nn.Linear(84, 50)
    self.fc3 = nn.Linear(50, 2)

  def forward(self, x):
    x = x.view(-1, 12288)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
simplenet = SimpleNet()

## **Create an Optimizer**

Training a network involves **passing data through the network**, using the loss function to determine the difference between prediction and the actual label, and then using that information to update the weights of the network in an attempt to make the loss function return as small a loss as possible.

Here, we use `Adam` as our optimizer with a learning rate: `lr`, of 0.001

In [ ]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## **Copy the model to GPU**

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else: 
  device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

## **Training**

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
  ### Train the model
  for epoch in range(1, epochs+1):
    training_loss = 0.0
    valid_loss = 0.0
    model.train()
    for batch in train_loader:
      # Optimizer zero grad
      optimizer.zero_grad()
      inputs, targets = batch
      inputs = inputs.to(device)
      targets = targets.to(device)
      # Forward pass
      output = model(inputs)
      # Calculate loss
      loss = loss_fn(output, targets)
      # Loss backward (backpropagation)
      loss.backward()
      # Optimizer step (gradient descent)
      optimizer.step()
      training_loss += loss.data.item() * inputs.size(0)
    training_loss /= len(train_loader.dataset)

    ### Evaluate the model on the test set
    model.eval()                          
    num_correct = 0
    num_examples = 0
    for batch in val_loader:
      inputs, targets = batch
      inputs = inputs.to(device)
      # Forward pass
      output = model(inputs)
      targets = targets.to(device)
      # Calculate loss
      loss = loss_fn(output, targets)
      valid_loss += loss.data.item() * inputs.size(0)
      correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
      num_correct += torch.sum(correct).item()
      num_examples += correct.shape[0] 
    valid_loss /= len(val_loader.dataset)

    print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'
    .format(epoch, training_loss, valid_loss, num_correct / num_examples))

In [ ]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), 
      train_data_loader, val_data_loader, epochs=5, device=device)

Epoch: 1, Training Loss: 1.61, Validation Loss: 7.58, accuracy = 0.22
Epoch: 2, Training Loss: 3.03, Validation Loss: 0.94, accuracy = 0.74
Epoch: 3, Training Loss: 0.48, Validation Loss: 2.12, accuracy = 0.36
Epoch: 4, Training Loss: 1.01, Validation Loss: 0.83, accuracy = 0.65
Epoch: 5, Training Loss: 0.33, Validation Loss: 1.20, accuracy = 0.53


## **Making predictions**

In [ ]:
labels = ['cat','fish']

img = Image.open("/content/images/val/fish/100_1422.JPG") 
img = img_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction]) 

fish


## **Saving Models**

In [ ]:
torch.save(simplenet, "/content/simplenet")  # to save
simplenet = torch.load("/content/simplenet")  # to load a previously saved 

In [ ]:
torch.save(simplenet.state_dict(), "/content/simplenet")    # save that contains the maps of each layer's parameters in the model.
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/content/simplenet")
simplenet.load_state_dict(simplenet_state_dict)             # assigns parameters to layers in the model that do exist

<All keys matched successfully>